In [ ]:
knitr::opts_chunk$set(
  collapse = TRUE,
  comment = "#>"
)

In [ ]:
# Install Google Colab dependencies
# Note: this can take 30+ minutes (many of the dependencies include C++ code, which needs to be compiled)

# First install `sf`, `ragg` and `textshaping` and their system dependencies:
system("apt-get -y update && apt-get install -y  libudunits2-dev libgdal-dev libgeos-dev libproj-dev libharfbuzz-dev libfribidi-dev")
install.packages("sf")
install.packages("textshaping")
install.packages("ragg")

# Install system dependencies of some other R packages that Voyager either imports or suggests:
system("apt-get install -y libfribidi-dev libcairo2-dev libmagick++-dev")

# Install Voyager from Bioconductor:
install.packages("BiocManager")
BiocManager::install(version = "3.17", ask = FALSE, update = FALSE, Ncpus = 2)
BiocManager::install("scater")
system.time(
  BiocManager::install("Voyager", dependencies = TRUE, Ncpus = 2, update = FALSE)
)

# Additional dependencies for this notebook
system("apt-get install -y libfftw3-dev libprotobuf-dev")
BiocManager::install("EBImage")
BiocManager::install("terra")
BiocManager::install("rmapshaper")
install.packages("reticulate")

packageVersion("Voyager")

# Introduction
In a [more introductory vignette](https://pachterlab.github.io/voyager/articles/visium_10x.html), we performed basic non-spatial analyses on a mouse olfactory bulb Visium dataset from the 10X website. In this vignette, we perform spatial analyses in histological space as well as in gene expression space. 

Here we load the packages used in this vignette:

In [ ]:
library(Voyager)
library(SpatialFeatureExperiment)
library(SingleCellExperiment)
library(ggplot2)
library(scater)
library(scuttle)
library(scran)
library(stringr)
library(patchwork)
library(bluster)
library(rjson)
library(EBImage)
library(terra)
library(rlang)
library(sf)
library(rmapshaper)
library(dplyr)
library(BiocParallel)
library(BiocNeighbors)
library(reticulate)
theme_set(theme_bw())

In [ ]:
# Specify Python version to use gget
PY_PATH <- system("which python", intern = TRUE)
use_python(PY_PATH)
py_config()

In [ ]:
system("pip3 install gget")

In [ ]:
gget <- import("gget")

Here we download the data from the 10X website. This is the unfiltered gene count matrix:

In [ ]:
if (!file.exists("visium_ob.tar.gz"))
    download.file("https://cf.10xgenomics.com/samples/spatial-exp/2.0.0/Visium_Mouse_Olfactory_Bulb/Visium_Mouse_Olfactory_Bulb_raw_feature_bc_matrix.tar.gz", 
                  destfile = "visium_ob.tar.gz")

This is the spatial information:

In [ ]:
if (!file.exists("visium_ob_spatial.tar.gz"))
    download.file("https://cf.10xgenomics.com/samples/spatial-exp/2.0.0/Visium_Mouse_Olfactory_Bulb/Visium_Mouse_Olfactory_Bulb_spatial.tar.gz", 
                  destfile = "visium_ob_spatial.tar.gz")

Decompress the downloaded content:

In [ ]:
if (!dir.exists("outs")) {
    dir.create("outs")
    system("tar -xvf visium_ob.tar.gz -C outs")
    system("tar -xvf visium_ob_spatial.tar.gz -C outs")
}

Contents of the `outs` directory as from Space Ranger is explained in [the introductory vignette](https://pachterlab.github.io/voyager/articles/visium_10x.html).

Here we read the data into R as an SFE object.

In [ ]:
(sfe <- read10xVisiumSFE(samples = ".", type = "sparse", data = "raw"))

Here we add QC metrics, already plotted in the introductory vignette.

In [ ]:
is_mt <- str_detect(rowData(sfe)$symbol, "^mt-")

In [ ]:
sfe <- addPerCellQCMetrics(sfe, subsets = list(mito = is_mt))

# Tissue segmentation
While Space Ranger can automatically detect which spots are in tissue and the Loupe browser can be used to manually annotate which spots are in tissue, it may be interesting to get the tissue outline polygon, so we would know how much each spot overlaps with the tissue and plot the outline. The tissue boundary polygon can be manually annotated with QuPath, which saves the polygon as a GeoJSON and can be directly read into R with `st_read()`. 

Or we can segment the tissue computationally. R generally isn't great for image processing, but there are some packages that can perform the segmentation, such as [`EBImage`](https://bioconductor.org/packages/release/bioc/html/EBImage.html), which is based on its own in house C and C++ code, and [`imager`](https://github.com/dahtah/imager/), which is based on [`CImg`](http://cimg.eu/).

Here we don't have the full resolution image. We will perform tissue segmentation on the high resolution downsampled image and then scale it to make the coordinates of the tissue boundary match those of the spots. The `EBImage` package is used here. Compared to OpenCV, `EBImage` is slow on the full resolution image, but should be fine here for the downsized image.

In [ ]:
img <- readImage("outs/spatial/tissue_hires_image.png")
display(img)

When rendered as a static webpage, the image is static, but when run interactively, this image will be shown in an interactive widget where you can zoom and pan.

Here we show the RGB channels separately

In [ ]:
img2 <- img
colorMode(img2) <- Grayscale
display(img2, all = TRUE)

In [ ]:
hist(img)

The tissue can be discerned with thresholding. The tall peak on the right is the background. The much lower peaks from around 0.6 to 0.85 must be the tissue. To capture the faint bluish region, the blue channel is used for thresholding. The threshold here is chosen based on the histogram and experimenting with nearby values.

In [ ]:
mask <- img2[,,3] < 0.87
display(mask)

Then we use an opening operation (erosion followed by dilation) to denoise

In [ ]:
kern <- makeBrush(3, shape='disc')
mask_open <- opening(mask, kern)
display(mask_open)

There are some small holes in the tissue, which can be removed by a closing operation (dilation followed by erosion):

In [ ]:
mask_close <- closing(mask_open, kern)
display(mask_close)

There are some larger holes in the tissue mask, which may be real holes or faint regions with few nuclei missed by thresholding. They might not be large enough to affect which Visium spots intersect the tissue.

Now the main piece of tissue is clear. It must be the object with the largest area. However, there are two small pieces that should belong to the tissue at the top left. The debris and fiducials can be removed by setting all pixels in the mask outside the bounding box of the main piece to 0. Here we assign a different value to each contiguous object with `bwlabel()`, and use `computeFeatures.shape()` to find the area among other shape features (e.g. perimeter) of each object.

In [ ]:
mask_label <- bwlabel(mask_close)
fts <- computeFeatures.shape(mask_label)

In [ ]:
head(fts)

In [ ]:
summary(fts[,"s.area"])

In [ ]:
max_ind <- which.max(fts[,"s.area"])
inds <- which(as.array(mask_label) == max_ind, arr.ind = TRUE)
head(inds)

In [ ]:
row_inds <- c(seq_len(min(inds[,1])-1), seq(max(inds[,1])+1, nrow(mask_label), by = 1))
col_inds <- c(seq_len(min(inds[,2])-1), seq(max(inds[,2])+1, nrow(mask_label), by = 1))
mask_label[row_inds, ] <- 0
mask_label[,col_inds] <- 0
display(mask_label)

Then remove the small pieces that are debris.

In [ ]:
unique(as.vector(mask_label))

In [ ]:
fts2 <- fts[unique(as.vector(mask_label))[-1],]
fts2 <- fts2[order(fts2[,"s.area"], decreasing = TRUE),]

In [ ]:
plot(fts2[,1][-1], type = "l", ylab = "Area")

In [ ]:
head(fts2, 10)

Object number 797 is a piece of debris at the bottom left. The other pieces with area over 100 pixels are tissue. Since debris really is small bits of tissue, so the boundary between debris and tissue can be blurry. Here the two are distinguished by morphology on the H&E image and proximity to the main tissue.

In [ ]:
#display(mask_label == 797)

Here we remove the debris from the mask

In [ ]:
mask_label[mask_label %in% c(797, as.numeric(rownames(fts2)[fts2[,1] < 100]))] <- 0

Since most holes in the mask are faint regions of the tissue missed by thresholding, the holes will be filled

In [ ]:
mask_label <- fillHull(mask_label)

In [ ]:
display(paintObjects(mask_label, img, col=c("red", "yellow"), opac=c(1, 0.3)))

This segmentation process took a lot of manual oversight, in choosing the threshold, choosing kernel size and shape in the opening and closing operations, deciding whether to fill the holes, and deciding what is debris and what is tissue. 

# Convert tissue mask to polygon
Now we have the tissue mask, which we will convert to polygon. While OpenCV can directly perform the conversion, as there isn't a comprehensive R wrapper of OpenCV, this conversion is more convoluted in R. We first convert the `Image` object to a raster as implemented in `terra`, the core R package for geospatial raster data. Then `terra` can convert the raster to polygon. As this image is downsized, the polygon will look quite pixelated. To mitigate this pixelation and save memory, the `ms_simplify()` function is used to simplify the polygon, only keeping a small proportion of all vertices. The `st_simplify()` function in `sf` can also simplify the polygons, but we can't specify what proportion of vertices to keep.

In [ ]:
raster2polygon <- function(seg, keep = 0.2) {
    r <- rast(as.array(seg), extent = ext(0, nrow(seg), 0, ncol(seg))) |> 
        trans() |> flip()
    r[r < 1] <- NA
    contours <- st_as_sf(as.polygons(r, dissolve = TRUE))
    simplified <- ms_simplify(contours, keep = keep)
    list(full = contours,
         simplified = simplified)
}

In [ ]:
tb <- raster2polygon(mask_label)

Before adding the geometry to the SFE object, it needs to be scaled to match the coordinates of the spots

In [ ]:
scale_factors <- fromJSON(file = "outs/spatial/scalefactors_json.json")

In [ ]:
tb$simplified$geometry <- tb$simplified$geometry / scale_factors$tissue_hires_scalef

In [ ]:
tissueBoundary(sfe) <- tb$simplified

In [ ]:
plotSpatialFeature(sfe, "sum", annotGeometryName = "tissueBoundary", 
                   annot_fixed = list(fill = NA, color = "black"),
                   image_id = "lowres") +
    theme_void()

The mouse olfactory bulb is conventionally plotted horizontally. The entire SFE object can be transposed in histologial space to make the olfactory bulb horizontal.

In [ ]:
sfe <- SpatialFeatureExperiment::transpose(sfe)

In [ ]:
plotSpatialFeature(sfe, "sum", annotGeometryName = "tissueBoundary", 
                   annot_fixed = list(fill = NA, color = "black"),
                   image_id = "lowres")

Then we can use geometric operations to find which spots intersect tissue, which spots are covered by tissue, and how much of each spot intersects tissue.

In [ ]:
# Which spots intersect tissue
sfe$int_tissue <- annotPred(sfe, colGeometryName = "spotPoly", 
                            annotGeometryName = "tissueBoundary",
                            pred = st_intersects)
sfe$cov_tissue <- annotPred(sfe, colGeometryName = "spotPoly", 
                            annotGeometryName = "tissueBoundary",
                            pred = st_covered_by)

Discrepancies between Space Ranger's annotation and the annotation based on tissue segmentation here:

In [ ]:
sfe$diff_sr <- case_when(sfe$in_tissue == sfe$int_tissue ~ "same",
                         sfe$in_tissue & !sfe$int_tissue ~ "Space Ranger",
                         sfe$int_tissue & !sfe$in_tissue ~ "segmentation") |> 
    factor(levels = c("Space Ranger", "same", "segmentation"))
plotSpatialFeature(sfe, "diff_sr", 
                   annotGeometryName = "tissueBoundary", 
                   annot_fixed = list(fill = NA, size = 0.5, color = "black")) +
    scale_fill_brewer(type = "div", palette = 4)

Spots at the margin can intersect the tissue without being covered by it.

In [ ]:
sfe$diff_int_cov <- sfe$int_tissue != sfe$cov_tissue
plotSpatialFeature(sfe, "diff_int_cov", 
                   annotGeometryName = "tissueBoundary", 
                   annot_fixed = list(fill = NA, size = 0.5, color = "black"))

We can also get the geometries of the intersections between the tissue and the Visium spots, and then calculate what percentage of each spot is in tissue. However, this percentage may not be very useful if the tissue segmentation is subject to error. This percentage may be more useful for pathologist annotated histological regions or objects such as nuclei and myofibers.

In [ ]:
spot_ints <- annotOp(sfe, colGeometryName = "spotPoly", 
                     annotGeometryName = "tissueBoundary", op = st_intersection)
sfe$pct_tissue <- st_area(spot_ints) / st_area(spotPoly(sfe)) * 100

For spots that intersect tissue, does total counts relate to percentage of the spot in tissue?

In [ ]:
sfe_tissue <- sfe[,sfe$int_tissue]

In [ ]:
plotColData(sfe_tissue, x = "pct_tissue", y = "sum", subset = "diff_int_cov",
                name_true = "Not covered", name_false = "Covered", bins = 50)

Spots that are not fully covered by tissue have lower total UMI counts, which can be due to both that they are not fully in tissue and the cell types with lower total counts in the histological region near the edge, as some spots fully covered by tissue also have low UMI counts.

# Spatial autocorrelation of QC metrics

In [ ]:
colGraph(sfe_tissue, "visium") <- findVisiumGraph(sfe_tissue)

In [ ]:
qc_features <- c("sum", "detected", "subsets_mito_percent")

In [ ]:
sfe_tissue <- colDataUnivariate(sfe_tissue, "moran.mc", qc_features, nsim = 200)

In [ ]:
plotMoranMC(sfe_tissue, qc_features)

In [ ]:
sfe_tissue <- colDataUnivariate(sfe_tissue, "sp.correlogram", qc_features,
                                order = 8)

In [ ]:
plotCorrelogram(sfe_tissue, qc_features)

In [ ]:
sfe_tissue <- colDataUnivariate(sfe_tissue, "localmoran", qc_features)

In [ ]:
plotLocalResult(sfe_tissue, "localmoran", qc_features, ncol = 2,
                colGeometryName = "spotPoly", divergent = TRUE, 
                diverge_center = 0, image_id = "lowres", maxcell = 5e4)

In [ ]:
sfe_tissue <- colDataUnivariate(sfe_tissue, "LOSH", qc_features)

In [ ]:
plotLocalResult(sfe_tissue, "LOSH", qc_features, ncol = 2,
                colGeometryName = "spotPoly", image_id = "lowres", maxcell = 5e4)

In [ ]:
sfe_tissue <- colDataUnivariate(sfe_tissue, "moran.plot", qc_features)

In [ ]:
moranPlot(sfe_tissue, "subsets_mito_percent")

# Spatial autocorrelation of gene expression

Normalize data with the `scran` method, and find highly variable genes

In [ ]:
#clusters <- quickCluster(sfe_tissue)
#sfe_tissue <- computeSumFactors(sfe_tissue, clusters=clusters)
#sfe_tissue <- sfe_tissue[, sizeFactors(sfe_tissue) > 0]
sfe_tissue <- logNormCounts(sfe_tissue)

In [ ]:
dec <- modelGeneVar(sfe_tissue)
hvgs <- getTopHVGs(dec, n = 2000)

Find Moran's I for all highly variable genes:

In [ ]:
sfe_tissue <- runMoransI(sfe_tissue, features = hvgs, BPPARAM = MulticoreParam(2))

In [ ]:
plotRowDataHistogram(sfe_tissue, "moran_sample01")

The vast majority of genes have positive Moran's I. Here we'll find the genes with the highest Moran's I:

In [ ]:
top_moran <- rownames(sfe_tissue)[order(rowData(sfe_tissue)$moran_sample01, 
                                        decreasing = TRUE)[1:9]]

We can use the [gget info](https://pachterlab.github.io/gget/info.html) module from the [gget](https://pachterlab.github.io/gget/) package to get additional information on these genes, such as their descriptions, synonyms, transcripts and more from a collection of reference databases including [Ensembl](https://ensembl.org/), [UniProt](https://www.uniprot.org/) and [NCBI](https://www.ncbi.nlm.nih.gov/) Here, we are showing their gene descriptions from [NCBI](https://www.ncbi.nlm.nih.gov/):

In [ ]:
gget_info <- gget$info(top_moran)

rownames(gget_info) <- gget_info$primary_gene_name
select(gget_info, ncbi_description)

Plot the genes with the highest Moran's I:

In [ ]:
plotSpatialFeature(sfe_tissue, top_moran, ncol = 3, image_id = "lowres",
                   maxcell = 5e4, swap_rownames = "symbol")

Here global Moran's I seems to be more about tissue structure. 

Some genes have negative Moran's I that might not be statistically significant:

In [ ]:
neg_moran <- rownames(sfe_tissue)[order(rowData(sfe_tissue)$moran_sample01, 
                                        decreasing = FALSE)[1:9]]

In [ ]:
# Display NCBI descriptions for these genes
gget_info_neg <- gget$info(neg_moran)

rownames(gget_info_neg) <- gget_info_neg$primary_gene_name
select(gget_info_neg, ncbi_description)

In [ ]:
plotSpatialFeature(sfe_tissue, neg_moran, ncol = 3, swap_rownames = "symbol",
                   image_id = "lowres", maxcell = 5e4)

In [ ]:
sfe_tissue <- runUnivariate(sfe_tissue, "moran.mc", neg_moran, 
                            colGraphName = "visium", nsim = 200, alternative = "less")

In [ ]:
plotMoranMC(sfe_tissue, neg_moran, swap_rownames = "symbol")

In [ ]:
rowData(sfe_tissue)[neg_moran, c("moran_sample01", "moran.mc_p.value_sample01")]

As there are 2000 highly variable genes and 2000 tests, these would no longer be significant after correcting for multiple testing. 

Does global Moran's I relate to gene expression level?

In [ ]:
sfe_tissue <- addPerFeatureQCMetrics(sfe_tissue)
names(rowData(sfe_tissue))

In [ ]:
plotRowData(sfe_tissue, x = "mean", y = "moran_sample01") +
    scale_x_log10() +
    annotation_logticks(sides = "b") +
    geom_density2d()

Genes that are more highly expressed overall tend to have higher Moran's I. 

# Apply spatial analysis methods to gene expression space
Spatial statistics that require a spatial neighborhood graph can also be applied to the k nearest neighbor graph not in histological space but in gene expression space. This is done in more depth in [this vignette](https://pachterlab.github.io/voyager/articles/nonspatial.html).

In [ ]:
sfe_tissue <- runPCA(sfe_tissue, ncomponents = 30, subset_row = hvgs,
                     scale = TRUE) # scale as in Seurat

In [ ]:
foo <- findKNN(reducedDim(sfe_tissue, "PCA")[,1:10], k=10, BNPARAM=AnnoyParam())
# Split by row
foo_nb <- asplit(foo$index, 1)
dmat <- 1/foo$distance
# Row normalize the weights
dmat <- sweep(dmat, 1, rowSums(dmat), FUN = "/")
glist <- asplit(dmat, 1)
# Sort based on index
ord <- lapply(foo_nb, order)
foo_nb <- lapply(seq_along(foo_nb), function(i) foo_nb[[i]][ord[[i]]])
class(foo_nb) <- "nb"
glist <- lapply(seq_along(glist), function(i) glist[[i]][ord[[i]]])

listw <- list(style = "W",
              neighbours = foo_nb,
              weights = glist)
class(listw) <- "listw"
attr(listw, "region.id") <- colnames(sfe_tissue)

In [ ]:
colGraph(sfe_tissue, "knn10") <- listw

In [ ]:
sfe_tissue <- runMoransI(sfe_tissue, features = hvgs, BPPARAM = MulticoreParam(2),
                         colGraphName = "knn10", name = "moran_ns")

Here we store the results in "moran_ns", not to be confused with spatial Moran's I results.

These are the genes that tend to be more similar to their neighbors in the 10 nearest neighbor graph in PCA space for gene expression rather than in histological space:

In [ ]:
top_moran2 <- rownames(sfe_tissue)[order(rowData(sfe_tissue)$moran_ns_sample01, 
                                        decreasing = TRUE)[1:9]]

In [ ]:
# Display NCBI descriptions for these genes
gget_info2 <- gget$info(top_moran2)

rownames(gget_info2) <- gget_info2$primary_gene_name
select(gget_info2, ncbi_description)

In [ ]:
plotSpatialFeature(sfe_tissue, top_moran2, ncol = 3, swap_rownames = "symbol",
                   image_id = "lowres", maxcell = 5e4)

Although this Moran's I was not computed in histological space, these genes with the highest Moran's I in PCA space also show spatial structure, as different cell types reside in different spatial regions.

# Session info

In [ ]:
sessionInfo()